In [95]:
from rouge import Rouge
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd

In [56]:
tqdm.pandas()

In [7]:
DATA_PATH = "../data/results-gpt4o-mini.csv"
model_name = "multi-qa-mpnet-base-dot-v1"

## Getting the data

In [3]:
df = pd.read_csv(DATA_PATH)

We will use only the first 300 documents:

In [5]:
df = df.iloc[:300]

## Q1. Getting the embeddings model

In [8]:
embedding_model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Create the embeddings for the first LLM answer:

In [11]:
answer_llm = df.iloc[0].answer_llm
embedded_answer_llm = embedding_model.encode(answer_llm)

What's the first value of the resulting vector?

In [13]:
embedded_answer_llm[0]

-0.42244673

## Q2. Computing the dot product

In [46]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [57]:
df['dot_product'] = df[['answer_llm', 'answer_orig']].progress_apply(
    lambda x: embedding_model.encode(x[0]) @ embedding_model.encode(x[1]),
    axis=1
)

  0%|          | 0/300 [00:00<?, ?it/s]

/var/folders/4r/r614z2d5519_6qc_7yzz01pr0000gp/T/ipykernel_22210/3214321776.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda x: embedding_model.encode(x[0]) @ embedding_model.encode(x[1]),
/var/folders/4r/r614z2d5519_6qc_7yzz01pr0000gp/T/ipykernel_22210/3214321776.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dot_product'] = df[['answer_llm', 'answer_orig']].progress_apply(


What's the 75% percentile of the score?

In [59]:
df['dot_product'].describe()

count    300.000000
mean      27.495996
std        6.384742
min        4.547923
25%       24.307844
50%       28.336872
75%       31.674311
max       39.476013
Name: dot_product, dtype: float64

## Q3. Computing the cosine

In [92]:
df['cosine'] = df[['answer_llm', 'answer_orig']].progress_apply(
    lambda x: (
        lambda v1, v2: v1 / np.linalg.norm(v1) @ v2 / np.linalg.norm(v2)
    )(embedding_model.encode(x[0]), embedding_model.encode(x[1])),
    axis=1
)

  0%|          | 0/300 [00:00<?, ?it/s]

/var/folders/4r/r614z2d5519_6qc_7yzz01pr0000gp/T/ipykernel_22210/498542250.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  )(embedding_model.encode(x[0]), embedding_model.encode(x[1])),
/var/folders/4r/r614z2d5519_6qc_7yzz01pr0000gp/T/ipykernel_22210/498542250.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cosine'] = df[['answer_llm', 'answer_orig']].progress_apply(


What's the 75% cosine in the scores?

In [94]:
df['cosine'].describe()

count    300.000000
mean       0.728393
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
Name: cosine, dtype: float64

## Q4. Rouge

In [97]:
df.iloc[10]

answer_llm     Yes, all sessions are recorded, so if you miss...
answer_orig    Everything is recorded, so you won’t miss anyt...
document                                                5170565b
question                    Are sessions recorded if I miss one?
course                                 machine-learning-zoomcamp
dot_product                                            32.344711
cosine                                                  0.777956
Name: 10, dtype: object

In [98]:
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(df.iloc[10]['answer_llm'], df.iloc[10]['answer_orig'])[0]

What's the F score for rouge-1?

In [100]:
scores['rouge-1']['f']

0.45454544954545456

## Q5. Average rouge score

In [105]:
np.mean([score['f'] for score in scores.values()])

0.35490034990035496

## Q6. Average rouge score for all the data points

In [109]:
df['rouge_2_f'] = df[['answer_llm', 'answer_orig']].apply(
    lambda x: rouge_scorer.get_scores(x[0], x[1])[0]['rouge-2']['f']
)

/var/folders/4r/r614z2d5519_6qc_7yzz01pr0000gp/T/ipykernel_22210/696384798.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rouge_2_f'] = df[['answer_llm', 'answer_orig']].apply(


In [110]:
df

,answer_llm,answer_orig,document,question,course,dot_product,cosine,rouge_2_f
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp,17.515991,0.506754,NaN
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp,13.418400,0.388549,NaN
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp,25.313255,0.718599,NaN
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp,12.147417,0.337266,NaN
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp,18.747736,0.521792,NaN
...,...,...,...,...,...,...,...,...
295,An alternative way to load the data using the ...,Above users showed how to load the dataset dir...,8d209d6d,What is an alternative way to load the data us...,machine-learning-zoomcamp,34.001785,0.914175,NaN
296,You can directly download the dataset from Git...,Above users showed how to load the dataset dir...,8d209d6d,How can I directly download the dataset from G...,machine-learning-zoomcamp,33.690861,0.902190,NaN
297,You can fetch data for homework using the `req...,Above users showed how to load the dataset dir...,8d209d6d,Could you share a method to fetch data for hom...,machine-learning-zoomcamp,34.491524,0.904734,NaN
298,If the status code is 200 when downloading dat...,Above users showed how to load the dataset dir...,8d209d6d,What should I do if the status code is 200 whe...,machine-learning-zoomcamp,27.538353,0.726782,NaN
